In [ ]:
import os
import json
import pandas as pd
import traceback


In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=KEY)

In [ ]:
llm

In [ ]:
from langchain.chat_models import ChatGooglePalm
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    }
}

In [ ]:
TEMPLATE = """
Text:{text}
You are an expert mcq maker. Given above the text, it is your job to \
create a quiz of {number} multiple questions for {subject} students in {tone} tone.
Make sure to format your response like RESPONSE_JSON below and use it as guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number","subject","tone","response_json"],
    template=TEMPLATE
)

In [ ]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt,output_key="quiz",verbose=True)

In [ ]:
TEMPLATE2 = """
You are an expert grammerian and writer. Given a multiple choice question for {subject} students.\
you need to evaluate the complexicity of the question and give a complete analysis of the quiz. only use at max 50 words for complexicity
if the quiz is not at par with the cognitive and analytical abilities of the student,\
update the quiz questions which needs to be changed and the change the tone such that it perfectly fit the students ability.\
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [ ]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [ ]:
generate_evaluate_chain = SequentialChain(
    chains = [quiz_chain,review_chain],
    input_variables = ["text", "number","subject","tone","response_json"],
    output_variables = ["quiz","review"],
    verbose = True
)

In [ ]:
file_path = 'D:\Projects\MCQ_Generator\data.txt'

In [ ]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [ ]:
print(TEXT)

In [ ]:
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER = 5
SUBJECT = "electronic engineering"
TONE = "tough"

In [ ]:
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)

In [ ]:
response

In [ ]:
print(type(response))

In [ ]:
quiz = response.get("quiz")

In [ ]:
quiz_string = quiz.replace("```json", "").replace("```", "").strip()

In [ ]:
print(quiz_string)

In [ ]:
quiz = json.loads(quiz_string)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

print(f"Number of items in quiz_table_data: {len(quiz_table_data)}")

In [ ]:
quiz_table_data

In [ ]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("electronics.csv",index= False)